In [ ]:
import pandas as pd
import csv
import sys
from tqdm import tqdm

In [ ]:
TAGS_DIR = "../Data/Tags.csv"
QNS_DIR = "../Data/Questions.csv"

In [ ]:
tags_df = pd.read_csv(TAGS_DIR, sep=',')
print(tags_df.head())

In [ ]:
top_tag_map = tags_df.groupby('Tag', as_index=False).agg(lambda x: set(x))
top_tag_map

In [ ]:
qn_tag_map = tags_df.groupby('Id', as_index=True).agg(lambda x: set(x))
qn_tag_map

In [ ]:
top_tag_map["Tag Count"] = top_tag_map.apply(lambda row: len(row['Id']), axis = 1)

In [ ]:
top_tag_map_100 = top_tag_map.sort_values("Tag Count", ascending=False).head(100).reset_index(drop=True)
top_tag_map_100

In [ ]:
all_id = set()
for i in range(0, 100):
    print(top_tag_map_100.loc[i]['Tag'], len(top_tag_map_100.loc[i]['Id']))
    all_id.update(top_tag_map_100.loc[i]['Id'])
print(len(all_id))

In [ ]:
fin_count = 0
with open(QNS_DIR, "rt", newline='', encoding='ISO-8859-1') as qns_file:
    datareader = csv.DictReader(qns_file)
    try:
        for row in datareader:
            if(int(row["Id"]) in all_id and int(row["Score"]) >= 5):
                tags = list(qn_tag_map.at[int(row["Id"]), 'Tag'])
                fin_count+=1
    except Exception as e:
        print(e)
print(f"Total number of questions having top 100 tags and score of 5 or above: {fin_count}")

In [ ]:
def get_qns():
    with open(QNS_DIR, "rt", newline='', encoding='ISO-8859-1') as qns_file:
        datareader = csv.DictReader(qns_file)
        try:
            for row in datareader:
                if(int(row["Id"]) in all_id and int(row["Score"]) >= 5):
                    tags = list(qn_tag_map.at[int(row["Id"]), 'Tag'])
                    yield [row["Title"], row["Body"], tags, len(tags)]
        except:
            pass

In [ ]:
data = []
qn_generator = get_qns()

In [ ]:
data = []
with tqdm(total=fin_count, file=sys.stdout) as pbar:
    while(True):
        try:
            data.append(next(qn_generator))
            pbar.update(1)
        except StopIteration as e:
            break
        except Exception as e:
            print(e, file=sys.stderr)
            break
    df = pd.DataFrame(data, columns = ['Title', 'Body', 'Tags', 'Tag Count'])
    df.to_csv(f'StackSample_merged.csv', sep=";", encoding='utf-8-sig', index_label='ID')